In [1]:
from bs4 import BeautifulSoup
import requests
import time
from tqdm import tqdm

In [ ]:
## Download data - Note that this is commented out because it takes a while to complete, and I already have
## all the data downloaded so I don't want to accidnetally trigger this code again. 

urls = [
    "https://fbref.com/en/comps/9/2017-2018/schedule/2017-2018-Premier-League-Scores-and-Fixtures",
    "https://fbref.com/en/comps/9/2018-2019/schedule/2018-2019-Premier-League-Scores-and-Fixtures",
    "https://fbref.com/en/comps/9/2019-2020/schedule/2019-2020-Premier-League-Scores-and-Fixtures",
    "https://fbref.com/en/comps/9/2020-2021/schedule/2020-2021-Premier-League-Scores-and-Fixtures",
    "https://fbref.com/en/comps/9/2021-2022/schedule/2021-2022-Premier-League-Scores-and-Fixtures",
    "https://fbref.com/en/comps/9/2021-2022/schedule/2022-2023-Premier-League-Scores-and-Fixtures"   
]

htmls = [requests.get(url).text for url in urls]
soups = [BeautifulSoup(html) for html in htmls]
tables = [pd.read_html(html)[0] for html in htmls]

matchLinks = set()
for soup in soups:
    for link in soup.find_all("a"):
        href = "https://fbref.com" + str(link.get("href"))
        if href is not None and href.startswith("https://fbref.com/en/matches/") and href.endswith("Premier-League"):
            matchLinks.add(href)
matchLinks = list(matchLinks)

for i in range(len(urls)):
    tables[i].to_csv("raw-data/{}".format(urls[i].split("/")[-1]))

for matchLink in tqdm(matchLinks):
    time.sleep(5) # avoid getting blocked by fbref - overall takes about 3.5 hours to download everything
    name = matchLink.split("/")[-1]
    tmp = pd.read_html(matchLink)
    for index, table in enumerate(tmp):
        table.to_csv("raw-data/{}-table-{}".format(name, index))